In [1]:
import sys
sys.path.append('..')
from src.utils.llamaindex_retriever import LlamaIndexRetriever
from langchain.vectorstores import FAISS
from langchain.embeddings.azure_openai import AzureOpenAIEmbeddings
from langchain.embeddings.openai import OpenAIEmbeddings
from dotenv import load_dotenv
import os
from langchain.chat_models import AzureChatOpenAI
from typing import Any


def load_env_variables(file_path):
    load_dotenv(file_path)
    print("Environment variables loaded successfully!")

env_file_path = "../.env"
load_env_variables = load_env_variables(env_file_path)
max_tokens = 3500
temperature = 0.1

# embeddings = AzureOpenAIEmbeddings(azure_deployment=azure_deployment, openai_api_version=openai_api_version)
embeddings =  AzureOpenAIEmbeddings(
        deployment=os.getenv("EMB_DEPLOYMENT"),
        openai_api_version=os.getenv("EMB_OPENAI_API_VERSION"),
        model=os.getenv("EMB_MODEL"),
        openai_api_key=os.getenv("EMB_OPENAI_API_KEY"),
        openai_api_base=os.getenv("EMB_OPENAI_ENDPOINT"),
        openai_api_type=os.getenv("EMB_API_TYPE"),
    )

llm_service = AzureChatOpenAI(deployment_name=os.getenv('AZURE_OPENAI_DEPLOYMENT_NAME'), openai_api_version=os.getenv("OPENAI_API_VERSION"),
                        openai_api_base=os.getenv("OPENAI_API_BASE"), 
                        openai_api_type= os.getenv("OPENAI_API_TYPE"),
                        openai_api_key=os.getenv("OPENAI_API_KEY"),
                        max_tokens=max_tokens,
                        temperature=temperature)

Environment variables loaded successfully!


/Users/k.abhishek/anaconda3/envs/agentgpt/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `AzureOpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import AzureOpenAIEmbeddings`.
  warn_deprecated(
/Users/k.abhishek/anaconda3/envs/agentgpt/lib/python3.11/site-packages/langchain_community/embeddings/azure_openai.py:113: UserWarning: As of openai>=1.0.0, Azure endpoints should be specified via the `azure_endpoint` param not `openai_api_base` (or alias `base_url`). Updating `openai_api_base` from https://polarisopenai.openai.azure.com/ to https://polarisopenai.openai.azure.com//openai.
  warnings.warn(
/Users/k.abhishek/anaconda3/envs/agentgpt/lib/python3.11/site-packages/langchain_community/embeddings/azure_openai.py:1

In [2]:
#define utils
from src.utils.cube_semantic_custom import CubeSemanticLoader
def fetch_cube_metadata(*args, **kwargs):
    try:
        # # Load document from Cube meta api
        loader = CubeSemanticLoader(os.getenv("CUBE_API_URL"), os.getenv("CUBE_TOKEN"), False)
        documents = loader.load()
        # to_json()
        return documents
    except Exception as e:
        # Handle exceptions gracefully and return an error response
        print("Error in fetching metadata from cube: " + str(e))
        return 0

def create_vector_store(documents, local_vector_store_path, *args, **kwargs):
    print("Loaded documents: " + str(documents))
    vectorstore = FAISS.from_documents(documents, embeddings)
    vectorstore.save_local(local_vector_store_path)
    print("Vector store created and saved successfully!")

def load_vector_store(vector_store_path, embeddings, *args, **kwargs):
    # Load the vector store from the local file system
    vectorstore = FAISS.load_local(vector_store_path, embeddings, allow_dangerous_deserialization=True)
    print("Vector store loaded successfully!")
    
    return vectorstore

In [3]:
#load existing vector store
vector_store_path = "/Users/k.abhishek/Documents/experiments/metric_store/metric_store_gen_ai/data/vector_store/cube_meta_faiss_index"
vectorstore = load_vector_store(vector_store_path, embeddings)

Vector store loaded successfully!


Tools 

In [30]:
import json
from crewai import Agent, Task, Crew

from crewai.telemetry import Telemetry

def noop(*args, **kwargs):
    print("Telemetry method called and noop'd\n")
    pass
for attr in dir(Telemetry):
    if callable(getattr(Telemetry, attr)) and not attr.startswith("__"):
        setattr(Telemetry, attr, noop)

os.environ["OTEL_SDK_DISABLED"] = "true"

from langchain.tools import tool
from src.utils.llamaindex_retriever import LlamaIndexRetriever
from typing import Optional, Type
from langchain.callbacks.manager import (
    AsyncCallbackManagerForToolRun,
    CallbackManagerForToolRun,
)
# Import things that are needed generically
from langchain.pydantic_v1 import BaseModel, Field
# from langchain.tools import BaseTool, StructuredTool, tool
from crewai_tools import BaseTool

def get_similar_documents_faiss(query, max_number_documents=3):
  vectorstore = FAISS.load_local(vector_store_path, embeddings, allow_dangerous_deserialization=True)
  docs = vectorstore.similarity_search_with_relevance_scores(query, max_number_documents)
  relevant_documents = []
  for doc in docs:
      doc = doc[0]
      meta = {'text':doc.page_content, 'table_metadata': doc.metadata}
      relevant_documents.append(meta)
  return relevant_documents

def get_similar_documents(query, max_number_documents=3):
    return get_similar_documents_faiss(query, max_number_documents)


class QueryInput(BaseModel):
    query: str = Field(description="should be enquiry query")

class RephraseInputQuery(BaseTool):
    name:str = "rephrase_input_query"
    description :str = "Useful to rephrase the query to capture the intent of the user regarding metric information"
    args_schema: Type[BaseModel] = QueryInput

    def _run(
        self, query: str, run_manager: Optional[CallbackManagerForToolRun] = None
    ) -> str:
        """Use the tool."""
        metric_description = self.rephrase_input_query(query)
        return metric_description

    async def _arun(
        self, query: str, run_manager: Optional[AsyncCallbackManagerForToolRun] = None
    ) -> str:
        """Use the tool asynchronously."""
        raise NotImplementedError("custom_search does not support async")
    
    
    def rephrase_input_query(self, query, *args, **kwargs):
        agent = Agent(
                role='Intent Capturer',
                goal=
                'Rephrasing the query to capture the intent of the user regarding metric information',
                backstory=
                "You are an expert to understand the user's intent and rephrase the query to capture the intent of the user accurately.",
                llm = llm_service,
                allow_delegation=False)
        task = Task(
                agent=agent,
                description=
                f'Rephrase the query to capture the intent of the user regarding metric information. The query is {query}. Donot add any noise to the response',
                expected_output="some string",
            )
        extracted_metrics = task.execute()

        return extracted_metrics
    

    
class MetricDiscovery(BaseTool):
    name :str = "metric_discovery"
    description:str = """Useful for general user questions related to discovery, explaination, description, interpretation of metrics/measures/KPIs, tables or columns."""
    args_schema: Type[BaseModel] = QueryInput

    def _run(
        self, query: str, run_manager: Optional[CallbackManagerForToolRun] = None
    ) -> str:
        """Use the tool."""
        metric_description = self.metric_discovery(query)
        return metric_description

    async def _arun(
        self, query: str, run_manager: Optional[AsyncCallbackManagerForToolRun] = None
    ) -> str:
        """Use the tool asynchronously."""
        raise NotImplementedError("custom_search does not support async")
    
    
    def metric_discovery(self, query, *args, **kwargs):
        """Useful for general user questions related to discovery, explaination, description, interpretation of metrics/measures/KPIs, tables or columns."""
        relevant_documents = get_similar_documents(query)
        agent = Agent(
                role='Data Analyst Assistant',
                goal=
                'Empower users to understand and utilize data effectively. This includes helping them discover relevant metrics, interpreting their meaning',
                backstory=
                "The primary purpose is to bridge the gap between raw data and user comprehension, fostering a data-driven culture within the organization.",
                llm = llm_service,
                allow_delegation=False)  
        
        task = Task(
                agent=agent,
                description=
                """ You are responding to  question {metric_description} with answer from the Metadata provided to you as {relevant_documents}}.
                    Strictly answer the question with the information present in metadata only.
                    Respond with "Sorry, the query is out of scope." if the answer is not present in metadata and terminate further reasoning and prcoess.
                    """,
                expected_output="some string",
            )
        output = task.execute()

        return output




Tasks

In [31]:
class MetricDiscoveryTasks():
  def metric_isolation(self, agent, query):
    return Task(description=(f"""
        Rephrase the query to capture the intent of the user regarding metric information. The query is {query}. Donot add any noise to the response.
        {self.__tip_section()}"""),
      agent=agent,
      expected_output="Reformatted query to capture the intent of the user regarding metric information.",
    )
  
  def metric_discovery(self, agent):
    return Task(description=(f"""
        Answer the general user questions related to discovery, explaination, description, interpretation of metrics/measures/KPIs, tables or columns.
        {self.__tip_section()}"""),
      agent=agent,
      expected_output="If relevant metric exists in metadata, provide the answer. Else, respond with 'Sorry, the query is out of scope.' and terminate process.",
    )
  def __tip_section(self):
    return "If you do your BEST WORK, I'll give you a $10,000 commission!"
  
  

Agents

In [32]:
class MetricDiscoveryAgent():
  def user_intent_capture(self):
    return Agent(
      role='Intent Capturer',
      goal=
      'Rephrasing the query to capture the intent of the user regarding metric information',
      backstory=
      "You are an expert to understand the user's intent and rephrase the query to capture the intent of the user accurately.",
      verbose=True,
      tools=[
        RephraseInputQuery()
      ],
      llm = llm_service,
    )
  def discover_metric_info(self):
    return Agent(
     role='Data Analyst Assistant',
      goal=
      'Empower users to understand and utilize data effectively. This includes helping them discover relevant metrics, interpreting their meaning',
      backstory=
      "The primary purpose is to bridge the gap between raw data and user comprehension, fostering a data-driven culture within the organization.",
      llm = llm_service,
      verbose=True,
      tools=[
        MetricDiscovery()
      ],
    )

Experts

In [7]:
from abc import ABC, abstractmethod
class Expert(ABC):
    name: str = ""
    description: str = ""
    public_description: str = ""
    arg_description: str = "The argument to the function."

    @abstractmethod
    def call(
        self,
        query: str,
    ) -> str:
        pass

Expert definitions

In [8]:
from crewai.process import Process
class MetricDiscoveryInputCrew(Expert):
  name = "MetricDiscovery"
  description = (
       "Use this to answer the general user questions related to discovery, explaination, description, interpretation of metrics/measures/KPIs, tables or columns."
    )
  public_description = "Metric discovery crew"
  arg_description = "Query related to metrics."

  def crew_process(self, query: str):
    agents = MetricDiscoveryAgent()
    tasks = MetricDiscoveryTasks()
    # print(agents)
    # print(tasks)
    user_intent_capture_agent = agents.user_intent_capture()
    discover_metric_info_agent = agents.discover_metric_info()
    # print(metric_isolator_agent)
    # metric_isolator_task = tasks.metric_isolation(metric_isolator_agent, self.query)
    metric_isolator_task = tasks.metric_isolation(user_intent_capture_agent, query)
    metric_discover_task = tasks.metric_discovery(discover_metric_info_agent)
    
    # print("Metric_isolator_task", metric_isolator_task)
    crew = Crew(
      agents=[
        user_intent_capture_agent,
        discover_metric_info_agent,
      ],
      tasks=[
        metric_isolator_task,
        metric_discover_task
      ],
      verbose=False,
      process=Process.sequential,
    )
    return crew
  

  def run(self, query: str):
    crew = self.crew_process(query)
    result = crew.kickoff()
    return result
  
  def call(
        self, query: str, *args: Any, **kwargs: Any
    ) -> str:
    crew = self.crew_process(query)
    result = crew.kickoff()
    return result

In [9]:
#Available experts
from typing import Any, Dict, List, Optional, Union
from typing import Type, TypedDict

class ExpertDescription(TypedDict):
    """Representation of a callable expert"""
    name: str
    """The name of the expert."""
    description: str
    """A description of the expert."""
    parameters: dict[str, object]
    """The parameters of the expert."""

def get_expert_function(expert: Type[Expert]) -> ExpertDescription:
    """A function that will return the tool's function specification"""
    name = get_expert_name(expert)
    return {
        "name": name,
        "description": expert.description,
        "parameters": {
            "type": "object",
            "properties": {
                "reasoning": {
                    "type": "string",
                    "description": (
                        f"Reasoning is how the task will be accomplished with the current expert. "
                        "Detail your overall plan along with any concerns you have."
                        "Ensure this reasoning value is in the user defined langauge "
                    ),
                },
                "arg": {
                    "type": "string",
                    "description": expert.arg_description,
                },
            },
            "required": ["reasoning", "arg"],
        },
    }

def get_expert_name(expert: Type[Expert]) -> str:
    return expert.name

# def format_expert_name(expert_name: str) -> str:
#     return expert_name.lower()

def get_available_experts() -> List[Type[Expert]]:
    return [
        MetricDiscoveryInputCrew
    ]

query = "What is the most popular feature used by our paid subscribers?"
crew = MetricDiscoveryInputCrew()
result = crew.run(query)

Query breakdown into objectives/intent

In [10]:
from langchain import PromptTemplate
def create_sub_objectives_1(query):
    # Define the prompt template
    prompt_create_objectives = PromptTemplate(
        input_variables=["query"],
        template="""
    You are an intelligent assistant that helps users break down their goals into high-level, smaller objectives. The system you are part of serves as an interface between users and a database containing various metrics. You have access to metadata of these metrics, including table columns information.
    Goal: {query}
    Using the provided goal and any relevant chat history, break down the goal into high-level, smaller objectives whose intents are different. Make sure each objective is clear and distinct.
    ) """)
    prompt = prompt_create_objectives.format_prompt(query=query,)
    response = llm_service.invoke(str(prompt))
    return response.content

#############################################################################################################################################################################################
def complete(prompt, user_query):
    """
    Completes the prompt with the user objective.

    Args:
        prompt: The prompt template.
        user_objective: The user's objective.

    Returns:
        The completed prompt.
    """
    return prompt.format(query=user_query)

def create_sub_objectives_2(query):
    prompt_objective_breakdwon = PromptTemplate.from_template("""
    You are an intelligent assistant whose only and only goal is to break down user's query {query} into high-level, smaller objectives in form of list. You serve as an interface between users and a database containing various metrics/KPIs/measures.
    Strictly note that it is not necessary to break down the query in all the following intents. The intents are only for reference. If the query is straightforward and aligns with any one of the intent, then do not change the query. 
    Try to minimize the number of objectives and keep them as high-level as possible. 
    Each smaller objective should strictly fall into one of the following intent:
    Discover Metrics:** If you'd like to measure something specific, I can identify relevant metrics. For example, are you interested in suggest some metrics based on objective?
    Fetch Data:** If you already know the metric you're interested in, I can retrieve the data from the database.
    Interpret Metrics:** Once you have the data, I can explain what it means in the context of your objective. 
    Understand Metrics:** If you're unsure about a metric's definition or purpose, I can provide more information.

    Don't add noise to the response.
    Respond with a list.
    However, if the query involves something else entirely, like action outside current intents, give empty list.""")
    completed_prompt = complete(prompt_objective_breakdwon, query)
    response = llm_service.invoke(completed_prompt)
    return response.content

#############################################################################################################################################################################################
intents = {
    "discover_metrics": "Identify relevant metrics based on user objectives. (e.g., Suggest metrics to measure customer satisfaction)",
    "fetch_metric": "Retrieve data for a specific metric the user has already identified.",
    # "analyze_trends": "Analyze trends within the fetched data to provide deeper insights.",
    "interpret_metrics": "Explain the meaning of the data in the context of the user's objective.",
    "understand_metrics": "Provide information about a metric's definition and purpose."
}
def create_sub_objectives_3(query):
    prompt_objective_breakdwon = PromptTemplate.from_template("""You are an intelligent assistant focused on helping users with metrics and data. Your primary goal is to break down user queries ({query}) into a concise list of high-level objectives.
    **Intents:**

    {intents}

    Important Notes:
    * Alignment with Intents: The new objectives should strictly align with the provided intents.
    * Prioritize Clarity & Efficiency:  Minimize the number of objectives and maintain a high-level overview. 
    * Direct Match: If the query aligns perfectly with one intent (e.g., "What's the current customer churn rate?" for Fetch Data), don't alter it.
    * Outside Scope: If the query falls outside these intents (e.g., "Book a flight"), return an empty list.
    * Focus on User Need: Keep responses clear and concise, avoiding unnecessary information. 
    * Output Format:  Respond with a list containing the breakdown of objectives.

    **Here's the user query: {query}

    What is the high-level breakdown of objectives for this query?
    """)

    # Format the intent descriptions for the prompt
    formatted_intents = "\n".join([f"* {intent_name}: {description}" for intent_name, description in intents.items()])
    query = "What dose the vessel name column represents?"
    # Insert formatted intents into the prompt template
    completed_prompt = prompt_objective_breakdwon.format(intents=formatted_intents, query=query)
    # Use the completed prompt with the AzureChatOpenAI model
    response = llm_service.invoke(completed_prompt)
    return response.content   

#############################################################################################################################################################################################

def create_sub_objectives(query):
    response = create_sub_objectives_3(query)
    return response

In [11]:
import ast
import re
from typing import List, TypeVar
from langchain.schema import BaseOutputParser, OutputParserException

T = TypeVar("T")

class TaskOutputParser(BaseOutputParser[List[str]]):
    """
    Extension of LangChain's BaseOutputParser
    Responsible for parsing task creation output into a list of task strings
    """

    tasks: List[str] = []

    def __init__(self, *, tasks: List[str]):
        super().__init__()
        self.tasks = tasks

    def parse(self, text: str) -> List[str]:
        """
        Parses the task creation output into a list of task strings.

        Args:
            text (str): The task creation output text.

        Returns:
            List[str]: The list of parsed task strings.

        Raises:
            OutputParserException: If parsing fails.
        """
        try:
            array_str = extract_array(text)
            all_tasks = [
                remove_prefix(task) for task in array_str if real_tasks_filter(task)
            ]
            return [task for task in all_tasks if task not in self.tasks]
        except Exception as e:
            msg = f"Failed to parse tasks from completion '{text}'. Exception: {e}"
            raise OutputParserException(msg)

    def get_format_instructions(self) -> str:
        """
        Returns the format instructions for the task output.

        Returns:
            str: The format instructions.
        """
        return """
        The response should be a JSON array of strings. Example:

        ["Search the web for NBA news", "Write some code to build a web scraper"]

        This should be parsable by json.loads()
        """

def handle_multiline_string(input_str: str) -> List[str]:
    """
    Handles a multiline string as a list.

    Args:
        input_str (str): The input multiline string.

    Returns:
        List[str]: The processed lines as a list of strings.
    """
    processed_lines = [
        re.sub(r".*?(\d+\..+)", r"\1", line).strip()
        for line in input_str.split("\n")
        if line.strip() != ""
    ]

    if any(re.match(r"\d+\..+", line) for line in processed_lines):
        return processed_lines
    else:
        print(f"Failed to extract array from {input_str}")
        return [input_str]

def extract_array(input_str: str) -> List[str]:
    """
    Extracts an array from the input string.

    Args:
        input_str (str): The input string.

    Returns:
        List[str]: The extracted array as a list of strings.
    """
    regex = (
        r"\[\s*\]|"  # Empty array check
        r"(\[(?:\s*(?:\"(?:[^\"\\]|\\.)*\"|\'(?:[^\'\\]|\\.)*\')\s*,?)*\s*\])"
    )
    match = re.search(regex, input_str)
    if match is not None:
        return ast.literal_eval(match[0])
    else:
        return handle_multiline_string(input_str)

def remove_prefix(input_str: str) -> str:
    """
    Removes the prefix from the input string.

    Args:
        input_str (str): The input string.

    Returns:
        str: The input string with the prefix removed.
    """
    prefix_pattern = (
        r"^(Task\s*\d*\.\s*|Task\s*\d*[-:]?\s*|Step\s*\d*["
        r"-:]?\s*|Step\s*[-:]?\s*|\d+\.\s*|\d+\s*[-:]?\s*|^\.\s*|^\.*)"
    )
    return re.sub(prefix_pattern, "", input_str, flags=re.IGNORECASE)


def real_tasks_filter(input_str: str) -> bool:
    """
    Filters out non-task strings.

    Args:
        input_str (str): The input string.

    Returns:
        bool: True if the input string is a task, False otherwise.
    """
    no_task_regex = (
        r"^No( (new|further|additional|extra|other))? tasks? (is )?("
        r"required|needed|added|created|inputted).*"
    )
    task_complete_regex = r"^Task (complete|completed|finished|done|over|success).*"
    do_nothing_regex = r"^(\s*|Do nothing(\s.*)?)$"

    return (
        not re.search(no_task_regex, input_str, re.IGNORECASE)
        and not re.search(task_complete_regex, input_str, re.IGNORECASE)
        and not re.search(do_nothing_regex, input_str, re.IGNORECASE)
    )

def parse_with_handling(parser: BaseOutputParser[T], completion: str) -> T:
    """
    Parses the completion using the specified parser with error handling.

    Args:
        parser (BaseOutputParser[T]): The output parser.
        completion (str): The completion to parse.

    Returns:
        T: The parsed output.

    Raises:
        OutputParserException: If parsing fails.
    """
    try:
        return parser.parse(completion)
    except OutputParserException as e:
        raise e

def format_objectives(objectives: str):
    """
    Formats the objectives.

    Args:
        objectives (str): The objectives to format.

    Returns:
        The formatted objectives.
    """
    task_output_parser = TaskOutputParser(tasks=[])
    objectives = parse_with_handling(task_output_parser, objectives)
    return objectives

def get_sub_objectives_list(query):
    """
    Gets the list of sub-objectives for the given query.

    Args:
        query: The query to get sub-objectives for.

    Returns:
        The list of sub-objectives.
    """
    sub_objectives = create_sub_objectives(query)
    return format_objectives(sub_objectives)

Analyze the objective and assign an appropriate 

In [12]:
def get_available_experts_descritpion():
    """
    Retrieves the descriptions of available experts.

    Returns:
        expert_descriptions (dict): A dictionary containing expert names as keys and their descriptions as values.
    """
    available_experts = get_available_experts()
    experts = list(map(get_expert_function, available_experts))
    expert_descriptions = {expert['name'] : expert['description'] for expert in experts}
    return expert_descriptions

In [13]:
# Mapping of all the expert name to expert instance
def expertsName_to_ExpertInstance():
    """
    Returns a dictionary mapping expert names to expert instances.

    Returns:
        experts (dict): A dictionary where the keys are expert names and the values are expert instances.
    """
    available_experts = get_available_experts()
    experts = {expert.name : expert for expert in available_experts}
    return experts

In [34]:
def assign_objective_to_expert(query, sub_objective):
    """
    Assigns the given query and sub-objective to the best expert based on their expertise.

    Args:
        query (str): The high-level objective.
        sub_objective (str): The current task.

    Returns:
        expert_object: The selected expert object.

    Raises:
        None
    """
    expert_descriptions = get_available_experts_descritpion()

    # Define the prompt template for analyzing the task
    analyze_task_prompt = PromptTemplate(
        template="""
        High level objective: "{query}"
        Current task: "{sub_objective}"

        Based on this information, use the best expert to make progress or accomplish the task entirely. Each expert has a name and description of their expertise. 
        Select the correct expert by being smart and efficient. Ensure "reasoning" and only "reasoning" is in the english language.
        Following are the list of experts {experts}

        Note you MUST select a expert and return the name and reasoning of the expert in form of dictionary, for example "\ "name": selected_expert_name, "reasoning": ....\"

        """,
        input_variables=["query", "sub_objective", "experts"],
    )

    # Format the prompt with the given query, sub-objective, and expert descriptions
    analyze_prompt = analyze_task_prompt.format(query=query, sub_objective=sub_objective, experts=expert_descriptions)

    # Invoke the language model service to analyze the task prompt
    response = llm_service.invoke(analyze_prompt)

    # Parse the selected expert from the response
    selected_expert = json.loads(response.content)

    # Get the expert object based on the selected expert's name
    expert_object = expertsName_to_ExpertInstance().get(selected_expert['name'])

    return expert_object


In [ ]:

def main(query):
    """
    Executes the main flow of the agentic workflow.

    Args:
        query (str): The query to be processed.

    Returns:
        list: A list of results for each sub-objective.
    """
    # Get the list of sub-objectives
    sub_objectives = get_sub_objectives_list(query)

    complete_response = []
    for sub_objective in sub_objectives:
        # Assign the objective to an expert and create an instance
        expert = assign_objective_to_expert(query, sub_objective)
        expert_instance = expert()
        
        # Call the expert's method to process the query
        result = expert_instance.call(query)
        
        # Append the result to the complete response list
        complete_response.append(result)
    
    print("All the objectives are completed successfully!")
    return complete_response
    

In [27]:
def main(query):
    """
    Executes the main flow of the agentic workflow.

    Args:
        query (str): The query to be processed.

    Returns:
        list: A list of results for each sub-objective.
    """
    # Get the list of sub-objectives
    sub_objectives = get_sub_objectives_list(query)
    print("Sub objectives are: ", sub_objectives)

    complete_response = []
    for sub_objective in sub_objectives:
        print("Sub objective is: ", sub_objective)
        
        # Assign the objective to an expert and create an instance
        expert = assign_objective_to_expert(query, sub_objective)
        expert_instance = expert()
        
        # Call the expert's method to process the query
        result = expert_instance.call(query)
        print("Result is: ", result)
        
        # Append the result to the complete response list
        complete_response.append(result)
    
    print("All the objectives are completed successfully!")
    return complete_response

In [33]:
query = "What are the top three metrics that indicate the overall health of our business?"
output = main(query)

Failed to extract array from - understand_metrics: Clarify the definition and purpose of the "vessel name" column in the dataset.
Sub objectives are:  ['- understand_metrics: Clarify the definition and purpose of the "vessel name" column in the dataset.']
Sub objective is:  - understand_metrics: Clarify the definition and purpose of the "vessel name" column in the dataset.
{'name': 'MetricDiscovery', 'reasoning': "The 'vessel name' column likely refers to the name of a ship or maritime vessel. Understanding the purpose of this column involves interpreting the data it represents, which falls under the expertise of the MetricDiscovery expert. This expert is suited for explaining the definition and purpose of specific metrics or columns in a dataset."}
Telemetry method called and noop'd

Telemetry method called and noop'd

Telemetry method called and noop'd



> Entering new CrewAgentExecutor chain...
Thought: The user is asking for specific metrics that reflect the health of their busine